In [2]:
import os
import pickle, random
import numpy as np
import faiss
import tqdm
import json, tiktoken
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import difflib, requests
import openai
import random, time
import google.generativeai as palm

openai.api_key = "Your Token"

/Users/kamel/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Loading the embedding

We embedded the training and test sets using the OpenAI model. Then, we input these embeddings into FAISS to select the relevant examples for in-context learning.

In [3]:
filel = open('Embedding/PyPIBugs_embedding_Training', 'rb')
PyPiBugs_training = pickle.load(filel)

In [5]:
file = open('Embedding/PyPIBugs_embedding_Testing', 'rb')
PyPiBugs_testing = pickle.load(file)


# FAISS

We implemented FAISS to select the most similar examples from the training set for each test example.

In [6]:
d = PyPiBugs_training.shape[1]
print(d)

1536


In [7]:
index = faiss.IndexFlatL2(d)

In [8]:
index.is_trained

True

In [9]:
index.add(PyPiBugs_training)

In [10]:
index.ntotal

3662

In [11]:
Idx = []
for i in tqdm.tqdm(range(len(PyPiBugs_testing))):
    k = 4
    xq = np.array(PyPiBugs_testing[i][0])
    xq = xq.reshape(1, 1536)
    D, I = index.search(xq, k)
    Idx.append(["Traning_ID", I, "Testing_ID", i])# , "Distance", D

100%|███████████████████████████████████████| 458/458 [00:00<00:00, 1314.27it/s]


### Loading the dataset


In [13]:
with open('PyPIBugs/train.jsonl', 'r') as json_file:    
    json_list_traning = list(json_file)
    
with open('PyPIBugs/test.jsonl', 'r') as json_file:    
    json_list_testing = list(json_file)

In [14]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [15]:
labeling = []
shot_MAX = 4

### Code-Pair Classification

In [ ]:
print('Starting at ',(len(labeling))*2)
for counter, index in enumerate(tqdm.tqdm((range(((len(labeling))*2), 458, 2)))):

    function_1 = json.loads(json_list_testing[index])
    function_2 = json.loads(json_list_testing[index+1])
    few_shot = '''For each pair of code snippets, identify the ones that contain bugs.
    The response should be either 'Code 1' or 'Code 2'. The answer must be precisely 'Code 1' or 'Code 2'.
    '''
        
    samples = []
    for index_ in Idx[index][1][0]:
        function = json_list_traning[index_]
        result = json.loads(function)
        func = result['func']
        samples.append(index_)
        
    
    duplicate = []
    shot = 0
    for f in samples:
        if f in duplicate:
            continue
        if shot == shot_MAX:
            break
        shot = + 1
        function = json_list_traning[f]
        duplicate.append(f)
        result = json.loads(function)
        func = result['func']
        Label = ("Code 1") if result["target"] == 0 else ("Code 2")
        
        if Label == "Code 1":
            function_pair = json_list_traning[f+1]
            duplicate.append(f+1)
            result_pair = json.loads(function_pair)
            func_pair = result_pair['func']
            Label_pair = ("Code 1") if result_pair["target"] == 0 else ("Code 2")
        else:
            function_pair = json_list_traning[f-1]
            duplicate.append(f-1)
            result_pair = json.loads(function_pair)
            func_pair = result_pair['func']
            Label_pair = ("Code 1") if result_pair["target"] == 0 else ("Code 2")
        
        if counter % 2 != 0:
            few_shot = few_shot + '\nCode 1: ' + func + '\n' + '\nCode 2: "' + func_pair + '"\n' + 'Label: Code 2' +  '\n### END ###\n'
        else:
            few_shot = few_shot + '\nCode 2: ' + func_pair + '\n' + '\nCode 1: "' + func + '"\n' + 'Label: Code 1' +  '\n### END ###\n'
        
    
    prompt_testing_example =  "\Code 1: " + str(function_1["func"]) + "\n" + "Code 2: " + str(function_2["func"]) + "\"" + "\nlabel: \n"

    prompt = few_shot + "\n" + prompt_testing_example
    
    
    MODEL = "gpt-3.5-turbo-16k"
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )
    #time.sleep(13)
    labeling.append(response.choices[0].message.content)
    #break

In [ ]:
Predict_label = []
count = 0
for i in labeling:
    if i[:6] == 'Code 2':
        count = count + 1
        Predict_label.append(1)
    else:
        Predict_label.append(0)
print(len(Predict_label))
Correct_label = [1] * int((len(Predict_label)))

print(len(Correct_label))
print(count)

In [ ]:
print('f1_score: ', round((f1_score(Correct_label, Predict_label) * 100),2))
print('accuracy_score: ',round((accuracy_score(Correct_label, Predict_label) * 100),2))

In [ ]:
print(classification_report(Correct_label, Predict_label))